## Loading the dataset
### Imports

In [58]:
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt
import re

### Convert filename to useful metadata

In [59]:
def filename_to_metadata(file_names: list[str], file_extension: str, only_labels=False):
    """
    Gets dataset type, fold, source name, label and take from .wav filename.

    :param file_names: List of .wav filename strings
    :param file_extension: The file extension (e.g. .wav).
    :param only_labels: Bool, where True means returning only a list of labels.
    :return: List of dictionaries ["dataset_type", "fold", "source_name", "label", "take"] or strings.
    """
    # stft-1-137-A-32.wav
    # stft --> Dataset type
    # 1 --> Fold
    # 137 --> Source name
    # A --> Take
    # 32 --> Label

    metadata_list = []

    for filename in file_names:
        filename = filename.replace(file_extension, "")
        dataset_type, fold, source_name, take, label = re.split(r"-", filename)

        data_dict = {"dataset_type": dataset_type, "fold": int(fold), "source_name": source_name, "take": take,
                     "label": int(label)}

        if only_labels:
            metadata_list.append(data_dict["label"])
        else:
            metadata_list.append(data_dict)

    return metadata_list

In [60]:
def filepath_to_img_data(path_to_dir: Path, filenames: list[str], resize_to_square=False):
    """
    Takes the path to images to create data objects.

    :param path_to_dir: Path object of the path to the data directory.
    :param filenames: List of filenames.
    :param resize_to_square: True means resizing the image to have the same width and height.
    :return:
    """
    img_data = []

    for filename in filenames:
        path_to_img = path_to_dir / filename

        base_img = cv2.imread(str(path_to_img))
        final_img = cv2.cvtColor(base_img, cv2.COLOR_BGR2RGB)

        if resize_to_square:
            new_img_size = max(final_img.shape)

            final_img = cv2.resize(final_img, (new_img_size, new_img_size))

        img_data.append(final_img)

    return img_data

### Relative paths

In [61]:
path_to_data = Path.cwd() / ".." / "data"
path_to_waveform = path_to_data / "waveform"

### Filenames of all images

In [62]:
#waveform_filenames = [file.name for file in path_to_waveform.glob("*.png")]
waveform_filenames = ["waveform-1-137-A-32.png"]

### Load images as dataset X and y

In [63]:
waveform_X = filepath_to_img_data(path_to_waveform, waveform_filenames)
waveform_y = filename_to_metadata(waveform_filenames, ".png", only_labels=True)